## Upload personal data to database
## ( the best version )

#### Import the libraries to get the env variables

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"

In [3]:
OPENAI_API_KEY 

'9ac347d13e834f288a2076ff9c7b418a'

#### Set the name of the database

In [4]:
database = "./conti_documents/"

#### Import the libraries to get the text and split it in chunks

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader

##### Get the text from the documents

In [7]:
# loader = DirectoryLoader(database, glob="./*.txt", loader_cls=TextLoader)
loader = DirectoryLoader(database, glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()
documents


[Document(page_content=' \n \n \nInternal   \n© Continental AG.  2023   CA 1014436  \nVersion  02 Page 1 (14) \nApproved  \n Location  Sibiu, Remote functions  Standard  Network Security  \nInternal   \n \n   \nLocation  \nStandard  \nNetwork Security  \n  \n \nResponsible Function  IT Infrastructure Team Leader  \nOrganizational Scope    Location Sibiu  \nReference (Superior Rule ) Continental Automotive InfoSec & CyberSec Community Wiki >  Continental \nAutomotive Information & Cyber Security >  Policies, Manuals, Procedures and \nMethods  \nFurther relevant Rules  - \nKey words  Security, Network , Checklist  \nFunctional contact  IT Infrastructure Engineer  \n  S-list Id. 16046461;    Decision date: 02 May 2023  09:22:53;    This document contains sensitive information;    Page: 1 of 15.', metadata={'source': 'conti_documents\\CA 1014436-02 SBZ Network Security .pdf', 'page': 0}),
 Document(page_content=' \n \n \nInternal   \n© Continental AG.  2023   CA 1014436  \nVersion  02 Page

##### Split the text into chunks

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
docs = text_splitter.split_documents(documents)

len(docs)

44

#### Import the libraries to connect to Chroma database

In [9]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [10]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


##### Upload the text in chroma database

In [11]:
import re

text = "conti_documents"
result = re.search(r'(?P<word>\w+)_', text)

if result:
    extracted_word = result.group('word')
    print(extracted_word)

database_name = extracted_word+"-database"
database_name

conti


'conti-database'

In [11]:
db2 = Chroma.from_documents(docs, embedding_function, persist_directory=database_name)